In [1]:
import numpy as np
import tqdm
from beir import util


/home/victor/Documents/travail_epita/venv_epita/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [18]:
import itertools
import random
import json
import os
from load_dataset import download_dataset


def create_smaller_corpus(corpus: dict, qrels: dict, folder_path: str, number_thrash: int):
	usefull_documents_title = list(
		dict.fromkeys(itertools.chain.from_iterable(map(lambda x: list(x.keys()), qrels.values()))))
	useless_documents_title = list(set(corpus.keys()) - set(usefull_documents_title))
	final_corpus_titles = random.sample(useless_documents_title, number_thrash) + usefull_documents_title
	final_corpus_titles = list(filter(lambda x: x in corpus.keys(), final_corpus_titles))
	simple_corpus = {k: corpus[k] for k in final_corpus_titles}
	list_json = []
	for el in tqdm.tqdm(simple_corpus):
		el_to_insert = simple_corpus[el]
		el_to_insert["_id"] = el
		list_json.append(el_to_insert)
	print("Saving corpus")
	with open(os.path.join(folder_path, "dbpedia-entity", "corpus.jsonl"), "w") as file:
		for item in tqdm.tqdm(list_json):
			json.dump(item,file)
			file.write("\n")
	return simple_corpus


is_clean_corpus = False
number_trash = 100000
corpus, queries, qrels = download_dataset("./data")
if is_clean_corpus:
	corpus = create_smaller_corpus(corpus, qrels, "./data", number_trash)

corpus

  0%|          | 0/200000 [00:00<?, ?it/s]

{'<dbpedia:Animalia_(book)>': {'text': "Animalia is an illustrated children's book by Graeme Base. It was originally published in 1986, followed by a tenth anniversary edition in 1996, and a 25th anniversary edition in 2012. Over three million copies have been sold.   A special numbered and signed anniversary edition was also published in 1996, with an embossed gold jacket.",
  'title': 'Animalia (book)'},
 '<dbpedia:Academy_Award_for_Best_Production_Design>': {'text': "The Academy Awards are the oldest awards ceremony for achievements in motion pictures. The Academy Award for Best Production Design recognizes achievement in art direction on a film. The category's original name was Best Art Direction, but was changed to its current name in 2012 for the 85th Academy Awards.  This change resulted from the Art Director's branch of the Academy being renamed the Designer's branch.",
  'title': 'Academy Award for Best Production Design'},
 '<dbpedia:An_American_in_Paris>': {'text': 'An Ameri

In [16]:
print("size of corpus is", len(corpus))
print("size of queries is", len(queries))
print("size of qrels is", len(qrels))

size of corpus is 107324
size of queries is 400
size of qrels is 400


In [4]:
corpus

{'<dbpedia:ASCII>': {'text': {'text': 'ASCII (/ˈæski/ ASS-kee), abbreviated from American Standard Code for Information Interchange, is a character-encoding scheme (the IANA prefers the name US-ASCII).  ASCII codes represent text in computers, communications equipment, and other devices that use text. Most modern character-encoding schemes are based on ASCII, though they support many additional characters.',
   'title': 'ASCII'},
  'title': 'ASCII'},
 '<dbpedia:Academy_Awards>': {'text': {'text': 'The Academy Awards or The Oscars is an annual American awards ceremony honoring cinematic achievements in the film industry. The various category winners are awarded a copy of a statuette, officially the Academy Award of Merit, which is better known by its nickname Oscar.',
   'title': 'Academy Awards'},
  'title': 'Academy_Awards'},
 '<dbpedia:Apollo>': {'text': {'text': 'Apollo (Attic, Ionic, and Homeric Greek: Ἀπόλλων, Apollōn (GEN Ἀπόλλωνος); Doric: Ἀπέλλων, Apellōn; Arcadocypriot: Ἀπείλω

We have 3 values.
-Corpus is a dictionary representing the description of each DBPedia page, representing an event, a person, a culture of anything else. Each element element contains the title of a page and the text associated.
-Queries represent a series of request by a user. It could be a question, a name, or a sentence describing what he looking for.
-Qrels represent the possible answers that match with queries. Each values is marked between 0 and 2. The best they have the best the response is.

In [5]:
list_corpus = [corpus[el]["text"] for el in tqdm.tqdm(corpus)]
list_corpus

100%|██████████| 140724/140724 [00:00<00:00, 1351190.59it/s]


[{'text': 'ASCII (/ˈæski/ ASS-kee), abbreviated from American Standard Code for Information Interchange, is a character-encoding scheme (the IANA prefers the name US-ASCII).  ASCII codes represent text in computers, communications equipment, and other devices that use text. Most modern character-encoding schemes are based on ASCII, though they support many additional characters.',
  'title': 'ASCII'},
 {'text': 'The Academy Awards or The Oscars is an annual American awards ceremony honoring cinematic achievements in the film industry. The various category winners are awarded a copy of a statuette, officially the Academy Award of Merit, which is better known by its nickname Oscar.',
  'title': 'Academy Awards'},
 {'text': 'Apollo (Attic, Ionic, and Homeric Greek: Ἀπόλλων, Apollōn (GEN Ἀπόλλωνος); Doric: Ἀπέλλων, Apellōn; Arcadocypriot: Ἀπείλων, Apeilōn; Aeolic: Ἄπλουν, Aploun; Latin: Apollō) is one of the most important and complex of the Olympian deities in classical Greek and Roman re

In [15]:
def dict_to_list_text(d: dict):
	ret = []
	for el in d:
		ret.append(d[el]["text"])
	return ret


corpus_list = dict_to_list_text(corpus)
corpus_list

[{'text': 'ASCII (/ˈæski/ ASS-kee), abbreviated from American Standard Code for Information Interchange, is a character-encoding scheme (the IANA prefers the name US-ASCII).  ASCII codes represent text in computers, communications equipment, and other devices that use text. Most modern character-encoding schemes are based on ASCII, though they support many additional characters.',
  'title': 'ASCII'},
 {'text': 'The Academy Awards or The Oscars is an annual American awards ceremony honoring cinematic achievements in the film industry. The various category winners are awarded a copy of a statuette, officially the Academy Award of Merit, which is better known by its nickname Oscar.',
  'title': 'Academy Awards'},
 {'text': 'Apollo (Attic, Ionic, and Homeric Greek: Ἀπόλλων, Apollōn (GEN Ἀπόλλωνος); Doric: Ἀπέλλων, Apellōn; Arcadocypriot: Ἀπείλων, Apeilōn; Aeolic: Ἄπλουν, Aploun; Latin: Apollō) is one of the most important and complex of the Olympian deities in classical Greek and Roman re

In [6]:
from sentence_transformers import SentenceTransformer
from requests.auth import HTTPBasicAuth

model_transformers = SentenceTransformer("bert-base-nli-mean-tokens")
#embeddings = model_transformers.encode()


In [12]:
import numpy as np
from sentence_transformers.util import cos_sim

sentences = ["I like you", "I love you", "I hate you", 'i regret you', "i do not like you"]
embedding = model_transformers.encode(sentences)
sim = np.zeros((len(sentences), len(sentences)))
for i in range(len(sentences)):
	sim[i:, i] = cos_sim(embedding[i], embedding[i:])
sim

array([[1.00000012, 0.        , 0.        , 0.        , 0.        ],
       [0.89882183, 1.00000012, 0.        , 0.        , 0.        ],
       [0.30012119, 0.32908571, 0.99999988, 0.        , 0.        ],
       [0.46551627, 0.44825739, 0.84543645, 0.99999988, 0.        ],
       [0.37578481, 0.33213681, 0.90139419, 0.84400928, 1.        ]])

In [ ]:
embedding.shape